In [1]:
import datetime as dt
import wget
import os
import pandas as pd
import numpy as np
import funcoes as f
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from keras.regularizers import l2
from keras.utils import load_img
from keras.utils import img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import absl.logging

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

absl.logging.set_verbosity(absl.logging.ERROR)
tf.get_logger().setLevel('WARNING')

In [2]:
dataset = "dp03" # "dp1" , "dp3" ou "dp03"
seed = np.random.randint(0, 9999)
batch_size = 32
n_samples = 1000

In [3]:
if dataset == "dp3":
    img = load_img('./teste sanidade/dp3/DP3_class0/image1.png')
elif dataset == "dp1":
    img = load_img('./teste sanidade/dp1/DP1_class0/image1.png')
elif dataset == "dp03":
    img = load_img('./teste sanidade/dp03/classe1/image1.png')
img = img_to_array(img)
img_shape = img.shape
del img
img_shape

(64, 64, 3)

In [4]:
dados = image_dataset_from_directory(f"./teste sanidade/{dataset}/",
                                                   label_mode="binary",
                                                  image_size=img_shape[:2],
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  seed=seed)

Found 2000 files belonging to 2 classes.


In [18]:
# pega 300 elementos do dataset para treino e 100 pra validação
dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
train_data = dados.take((n_samples//batch_size))
dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
validation_data = dados.take((100//batch_size))
print(len(train_data), len(validation_data))

31 3


# CNN hinge loss e regularizador

In [18]:
########## modelo csvm direto no keras

def CNN_hinge(img_shape=img_shape):
    # define our MLP network
    model = keras.Sequential()
    #model.add(layers.Input(shape=(20, 180, 3)))
    model.add(layers.Input(shape=img_shape))
    
    # reescala os dados pra entre 0 e 1
    model.add(layers.Rescaling(1./255))
    
    # primeira convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    # segunda convolucao
    #model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))#, padding="same"))
    #model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding="same"))
    #model.add(layers.MaxPool2D(pool_size=(2, 2)))
    #model.add(layers.Dropout(0.20))
    
    # terceira
    #model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding="same"))
    #model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding="same"))
    #model.add(layers.MaxPool2D(pool_size=(2, 2)))
    
    # FC_1024
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="linear"))#"relu")) # tirar o relu resolveu o problema de não convergir as vezes
    model.add(layers.Dense(8, activation="linear"))#"relu"))
    model.add(layers.Dense(1, kernel_regularizer=l2(0.01))) # aqui mandam usar l2 0,01
    model.add(layers.Activation('linear')) # aqui mandam por linear
    #model.add(layers.Activation('relu')) # tentar com tanh 
    
    return model

In [24]:
########## modelo cnn-svm direto no keras

model = CNN_hinge()
model.compile(loss = "categorical_hinge", optimizer="Adam", metrics=["accuracy",
                                                                     keras.metrics.Precision(),
                                                                     keras.metrics.Recall(), 
                                                                     keras.metrics.AUC()])

# callbacks
# salva o melhor modelo na pasta "modelo"
checkpoint = ModelCheckpoint("modelo", monitor='accuracy', verbose=0, save_best_only=True, mode='max')
# para de treinar se a acuracia de treino parar de aumentar por 3 epochs
es = EarlyStopping(monitor='accuracy', patience=4)
callbacks_list = [checkpoint, es]

#treina
history = model.fit(train_data,  epochs=25, use_multiprocessing=True, callbacks=callbacks_list )

# carrega o melhor modelo treinado e avalia
model = keras.models.load_model("modelo")
model.evaluate(validation_data)

Epoch 1/25
31/31 [==============================] - 2s 36ms/step - loss: 2.0145 - accuracy: 0.5379 - precision_24: 0.5447 - recall_24: 0.4037 - auc_24: 0.5190
Epoch 2/25
31/31 [==============================] - 1s 22ms/step - loss: 0.8813 - accuracy: 0.6532 - precision_24: 1.0000 - recall_24: 0.2758 - auc_24: 0.6575
Epoch 3/25
31/31 [==============================] - 1s 23ms/step - loss: 0.4747 - accuracy: 0.8014 - precision_24: 0.9528 - recall_24: 0.6247 - auc_24: 0.8918
Epoch 4/25
31/31 [==============================] - 0s 6ms/step - loss: 0.4190 - accuracy: 0.8014 - precision_24: 0.9390 - recall_24: 0.6473 - auc_24: 0.8772
Epoch 5/25
31/31 [==============================] - 1s 22ms/step - loss: 0.2205 - accuracy: 0.9032 - precision_24: 0.9791 - recall_24: 0.8287 - auc_24: 0.9807
Epoch 6/25
31/31 [==============================] - 1s 22ms/step - loss: 0.0215 - accuracy: 1.0000 - precision_24: 1.0000 - recall_24: 1.0000 - auc_24: 1.0000
Epoch 7/25
31/31 [=============================

[0.01794295571744442, 1.0, 1.0, 1.0, 1.0]

# CNN

In [17]:
def CNN(img_shape=img_shape):
    model = keras.Sequential()
    model.add(layers.Input(shape=img_shape))
    
    # reescala os dados pra entre 0 e 1
    model.add(layers.Rescaling(1./255))
    
    # convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    # classificação
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="linear"))#"relu"))
    model.add(layers.Dense(8, activation="linear"))#"relu"))
    # pra classificacao binaria parece que precisa usar sigmoid
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

In [27]:
model = CNN()
model.compile(loss = "binary_crossentropy", optimizer="Adam", metrics=["accuracy",
                                                                     keras.metrics.Precision(),
                                                                     keras.metrics.Recall(), 
                                                                     keras.metrics.AUC()])

# callbacks
# salva o melhor modelo na pasta "modelo"
checkpoint = ModelCheckpoint("modelo", monitor='accuracy', verbose=0, save_best_only=True, mode='max')
# para de treinar se a acuracia de treino parar de aumentar por 3 epochs
es = EarlyStopping(monitor='accuracy', patience=4)
callbacks_list = [checkpoint, es]

#treina
history = model.fit(train_data,  epochs=25, use_multiprocessing=True, callbacks=callbacks_list)

# carrega o melhor modelo treinado e avalia
model = keras.models.load_model("modelo")
model.evaluate(validation_data)

Epoch 1/25
31/31 [==============================] - 2s 29ms/step - loss: 2.3655 - accuracy: 0.5348 - precision_25: 0.5163 - recall_25: 0.4411 - auc_25: 0.5230
Epoch 2/25
31/31 [==============================] - 1s 22ms/step - loss: 0.6684 - accuracy: 0.6004 - precision_25: 0.5896 - recall_25: 0.6502 - auc_25: 0.6490
Epoch 3/25
31/31 [==============================] - 1s 22ms/step - loss: 0.4244 - accuracy: 0.9314 - precision_25: 0.9658 - recall_25: 0.8986 - auc_25: 0.9862
Epoch 4/25
31/31 [==============================] - 1s 28ms/step - loss: 0.2180 - accuracy: 0.9950 - precision_25: 0.9958 - recall_25: 0.9938 - auc_25: 0.9999
Epoch 5/25
31/31 [==============================] - 0s 6ms/step - loss: 0.1430 - accuracy: 0.9949 - precision_25: 0.9939 - recall_25: 0.9959 - auc_25: 0.9999
Epoch 6/25
31/31 [==============================] - 1s 21ms/step - loss: 0.0810 - accuracy: 1.0000 - precision_25: 1.0000 - recall_25: 1.0000 - auc_25: 1.0000
Epoch 7/25
31/31 [=============================

[0.06761527806520462, 1.0, 1.0, 1.0, 1.0]

# CSVM

Aqui preciso declarar os dados novamente com batch_size=1 pra poder extrair os dados mais facilmente

In [12]:
def CSVM(img_shape=img_shape):
    # define our MLP network
    model = keras.Sequential()
    #model.add(layers.Input(shape=(20, 180, 3)))
    model.add(layers.Input(shape=img_shape))
    model.add(layers.Rescaling(1./255))
    
    # primeira convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))#, padding="same"))
    #model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.15))
    model.add(layers.Flatten())
    
    return model

In [13]:
dados = image_dataset_from_directory(f"./teste sanidade/{dataset}/",
                                                   label_mode="binary",
                                                  image_size=img_shape[:2],
                                                  batch_size=1,
                                                  shuffle=True,
                                                  seed=seed)

Found 2000 files belonging to 2 classes.


In [14]:
# pega 300 elementos do dataset para treino e 100 pra validação
dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
train_data = dados.take((n_samples//batch_size)*batch_size)
dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
validation_data = dados.take((100//batch_size)*batch_size)
print(len(train_data), len(validation_data))

992 96


In [15]:
model = CSVM(img_shape=img_shape)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 30752)             0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
________________________________________________________

In [16]:
# coloca os dados de treino da SVM em um numpy array

predictions = np.empty(shape=(len(train_data), list(model.output.shape)[1]), dtype="float16")#, dtype="float32")
labels =  np.empty(len(train_data), dtype="int").reshape(-1, 1)
i = 0
j = 0
print("Preparando dados")
for x, y in train_data:
    #print(y)
    if i == j:
        print("\r", round(j/len(train_data)*100, 1),"%", end="    ")
        j += 1000
    predictions[i] = (model(x, training=False).numpy()[0])
    labels[i] = y.numpy()[0]
    i += 1
labels = np.ravel(labels)
print("\r", "100", "%", "         ")

Preparando dados
 100 %          


In [14]:
gamma = [2**x for x in range(-10,-3)]
gamma
# 0.001953125 melhor

[0.0009765625, 0.001953125, 0.00390625, 0.0078125, 0.015625, 0.03125, 0.0625]

In [15]:
# pega melhores parametros pro kernel rbf
from sklearn.model_selection import GridSearchCV
modelo_svm=SVC()
params = {"C":[0.1, 0.5, 1, 5, 10], "gamma": gamma}
grid_search = GridSearchCV(modelo_svm, params, n_jobs=-1)
grid_search.fit(predictions, labels)
grid_search.best_params_

{'C': 0.5, 'gamma': 0.0078125}

In [17]:
modelo_svm = LinearSVC()
#modleo_svm = SVC(kernel='rbf', gamma=1, C=0.01)
print("treinando modelo")
modelo_svm.fit(predictions, labels)
print("medindo score")
modelo_svm.score(predictions, labels)

treinando modelo
medindo score


D:\anaconda\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1.0

In [18]:
# dados de teste pro csvm
predictions = np.empty(shape=(len(validation_data), list(model.output.shape)[1]), dtype="float16")#, dtype="float32")
labels =  np.empty(len(validation_data), dtype="int").reshape(-1, 1)
i = 0
j = 0
print("Preparando dados")
for x, y in validation_data:
    #print(y)
    if i == j:
        print("\r", round(j/len(validation_data)*100, 1),"%", end="    ")
        j += 1000
    predictions[i] = (model(x, training=False).numpy()[0])
    labels[i] = y.numpy()[0]
    i += 1
labels = np.ravel(labels)
print("\r", "100", "%", "         ")

Preparando dados
 100 %          


In [19]:
print("testando modelo")
modelo_svm.score(predictions, labels)

testando modelo


1.0

## Treina multiplas vezes

In [19]:
repeticoes = 50
n_samples = 1000

scores_CNN_hinge = []
scores_CNN = []
scores_CSVM = []
resultado_linear = np.empty((repeticoes, 5))
resultado_poli = np.empty((repeticoes, 5))
resultado_rbf = np.empty((repeticoes, 5))
resultado_CNN_hinge = np.empty((repeticoes, 5))
resultado_CNN = np.empty((repeticoes, 5))

# treina n vezes e salva scores nos arrays
for repeticao in range(repeticoes):
    print(repeticao)
    #cria dados
    seed = np.random.randint(0, 9999)
    
    # precisa definir o checkpoint antes de começar cada CNN. Se não acaba usando um do outro
    checkpoint = ModelCheckpoint("modelo", monitor='accuracy', verbose=0, save_best_only=True, mode='max')
    # para de treinar se a acuracia de treino parar de aumentar por 3 epochs
    es = EarlyStopping(monitor='accuracy', patience=4)
    callbacks_list = [checkpoint, es]
    
    dados = image_dataset_from_directory(f"./teste sanidade/{dataset}/",
                                                   label_mode="binary",
                                                  image_size=img_shape[:2],
                                                  batch_size=batch_size,
                                                  shuffle=True,
                                                  seed=seed)
    
    # pega 300 elementos do dataset para treino e 100 pra validação
    dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
    train_data = dados.take((n_samples//batch_size))
    dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
    validation_data = dados.take((100//batch_size))
    
    # treina CNN hinge
    model = CNN_hinge()
    model.compile(loss = "categorical_hinge", optimizer="Adam", metrics=["accuracy",
                                                                     keras.metrics.Precision(),
                                                                     keras.metrics.Recall(), 
                                                                     keras.metrics.AUC()])
    model.fit(train_data,  epochs=25, use_multiprocessing=True, verbose=0, callbacks=callbacks_list)
    model = keras.models.load_model("modelo")
    resultado_CNN_hinge[repeticao] = model.evaluate(validation_data)
    del model
    
    checkpoint = ModelCheckpoint("modelo", monitor='accuracy', verbose=0, save_best_only=True, mode='max')
    # para de treinar se a acuracia de treino parar de aumentar por 3 epochs
    es = EarlyStopping(monitor='accuracy', patience=4)
    callbacks_list = [checkpoint, es]
    
    # treina CNN normal
    model = CNN()
    model.compile(loss = "binary_crossentropy", optimizer="Adam", metrics=["accuracy",
                                                                     keras.metrics.Precision(),
                                                                     keras.metrics.Recall(), 
                                                                     keras.metrics.AUC()])
    model.fit(train_data,  epochs=25, use_multiprocessing=True, verbose=0, callbacks=callbacks_list)
    model = keras.models.load_model("modelo")
    resultado_CNN[repeticao] = model.evaluate(validation_data)
    del model
    
    # prepara os dados pra CSVM
    dados = image_dataset_from_directory(f"./teste sanidade/{dataset}/",
                                                   label_mode="binary",
                                                  image_size=img_shape[:2],
                                                  batch_size=1,
                                                  shuffle=True,
                                                  seed=seed)

    dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
    train_data = dados.take((n_samples//batch_size)*batch_size)
    dados = dados.shuffle(len(dados), seed=seed, reshuffle_each_iteration=True)
    validation_data = dados.take((100//batch_size)*batch_size)

    model = CSVM(img_shape=img_shape)

    # coloca os dados de treino da SVM em um numpy array

    predictions = np.empty(shape=((n_samples//batch_size)*batch_size, list(model.output.shape)[1]), dtype="float16")#, dtype="float32")
    labels =  np.empty((n_samples//batch_size)*batch_size, dtype="int").reshape(-1, 1)
    i = 0
    j = 0
    #print("Preparando dados")
    for x, y in train_data:
        #print(y)
        if i == j:
            #print("\r", round(j/((300//batch_size)*batch_size)*100, 1),"%", end="    ")
            j += 1000
        predictions[i] = (model(x, training=False).numpy()[0])
        labels[i] = y.numpy()[0]
        i += 1
    labels = np.ravel(labels)
    #print("\r", "100", "%", "         ")
    
    
     # treina CSVM
    #modelo_svm = LinearSVC()
    modelo_linear = SVC(kernel='linear')
    modelo_poli = SVC(kernel='poly')
    modelo_rbf = SVC(kernel='rbf', C=0.5, gamma=0.0078125)

    modelo_linear.fit(predictions, labels)
    modelo_poli.fit(predictions, labels)
    modelo_rbf.fit(predictions, labels)
    
    # prepara dados de teste pro csvm
    predictions = np.empty(shape=(((100//batch_size)*batch_size), list(model.output.shape)[1]), dtype="float16")#, dtype="float32")
    labels =  np.empty(((100//batch_size)*batch_size), dtype="int").reshape(-1, 1)
    i = 0
    j = 0
    #print("Preparando dados")
    for x, y in validation_data:
        #print(y)
        if i == j:
            #print("\r", round(j/((100//batch_size)*batch_size)*100, 1),"%", end="    ")
            j += 1000
        predictions[i] = (model(x, training=False).numpy()[0])
        labels[i] = y.numpy()[0]
        i += 1
    labels = np.ravel(labels)
    #print("\r", "100", "%", "         ")
    
    del model
    
    y_pred = modelo_linear.predict(predictions)
    y_true = labels
    
    resultado_linear[repeticao] = np.array([accuracy_score(y_true, y_pred),
                   precision_score(y_true, y_pred), 
                   recall_score(y_true, y_pred), 
                   f1_score(y_true, y_pred),
                   roc_auc_score(y_true, y_pred)
                  ])
    
    y_pred = modelo_poli.predict(predictions)
    y_true = labels
    
    resultado_poli[repeticao] = np.array([accuracy_score(y_true, y_pred),
                   precision_score(y_true, y_pred), 
                   recall_score(y_true, y_pred), 
                   f1_score(y_true, y_pred),
                   roc_auc_score(y_true, y_pred)
                  ])
    
    y_pred = modelo_rbf.predict(predictions)
    y_true = labels
    
    resultado_rbf[repeticao] = np.array([accuracy_score(y_true, y_pred),
                   precision_score(y_true, y_pred), 
                   recall_score(y_true, y_pred), 
                   f1_score(y_true, y_pred),
                   roc_auc_score(y_true, y_pred)
                  ])

0
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 3ms/step - loss: 0.7200 - accuracy: 0.4375 - precision_1: 0.4375 - recall_1: 1.0000 - auc_1: 0.9365
Found 2000 files belonging to 2 classes.
1
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 4ms/step - loss: 0.0603 - accuracy: 1.0000 - precision_3: 1.0000 - recall_3: 1.0000 - auc_3: 1.0000
Found 2000 files belonging to 2 classes.
2
Found 2000 files belonging to 2 classes.


3/3 [==============================] - 0s 4ms/step - loss: 0.0187 - accuracy: 1.0000 - precision_4: 1.0000 - recall_4: 1.0000 - auc_4: 1.0000


3/3 [==============================] - 0s 9ms/step - loss: 0.1972 - accuracy: 1.0000 - precision_5: 1.0000 - recall_5: 1.0000 - auc_5: 1.0000
Found 2000 files belonging to 2 classes.
3
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 8ms/step - loss: 0.0846 - accuracy: 1.0000 - precision_7: 1.0000 - recall_7: 1.0000 - auc_7: 1.0000
Found 2000 files belonging to 2 classes.
4
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 6ms/step - loss: 0.6917 - accuracy: 0.5104 - precision_9: 0.0000e+00 - recall_9: 0.0000e+00 - auc_9: 0.7547
Found 2000 files belonging to 2 classes.
5
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 4ms/step - loss: 0.0341 - accuracy: 1.0000 - precision_11: 1.0000 - recall_11: 1.0000 - auc_11: 1.0000
Found 2000 files belonging to 2 classes.
6
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 4ms/step - loss: 0.1385 - accuracy: 1.0000 

46
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 4ms/step - loss: 0.0244 - accuracy: 1.0000 - precision_93: 1.0000 - recall_93: 1.0000 - auc_93: 1.0000
Found 2000 files belonging to 2 classes.
47
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 4ms/step - loss: 0.0442 - accuracy: 1.0000 - precision_95: 1.0000 - recall_95: 1.0000 - auc_95: 1.0000
Found 2000 files belonging to 2 classes.
48
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 1s 4ms/step - loss: 0.0676 - accuracy: 1.0000 - precision_97: 1.0000 - recall_97: 1.0000 - auc_97: 1.0000
Found 2000 files belonging to 2 classes.
49
Found 2000 files belonging to 2 classes.
3/3 [==============================] - 0s 4ms/step - loss: 0.1206 - accuracy: 1.0000 - precision_99: 1.0000 - recall_99: 1.0000 - auc_99: 1.0000
Found 2000 files belonging to 2 classes.


In [20]:
# transforma resultados da SVM em df
resultado_linear = pd.DataFrame(resultado_linear, columns=["accuracy", "precision", "recall", "f1-score", "AUC"])
resultado_poli = pd.DataFrame(resultado_poli, columns=["accuracy", "precision", "recall", "f1-score", "AUC"])
resultado_rbf = pd.DataFrame(resultado_rbf, columns=["accuracy", "precision", "recall", "f1-score", "AUC"]) 
# transforma resultados CNN em df e tira a loss
resultado_CNN = pd.DataFrame(resultado_CNN, columns=["loss", "accuracy", "precision", "recall", "AUC"]).drop(columns=["loss"])
resultado_CNN_hinge = pd.DataFrame(resultado_CNN_hinge, columns=["loss", "accuracy", "precision", "recall", "AUC"]).drop(columns=["loss"])
# calcula f1
resultado_CNN["f1-score"] = 2*(resultado_CNN["precision"]*resultado_CNN["recall"])/(resultado_CNN["precision"]+resultado_CNN["recall"])
resultado_CNN_hinge["f1-score"] = 2*(resultado_CNN_hinge["precision"]*resultado_CNN_hinge["recall"])/(resultado_CNN_hinge["precision"]+resultado_CNN_hinge["recall"])
# poe 0 nos nans
resultado_CNN.fillna(0, inplace=True)
resultado_CNN_hinge.fillna(0, inplace=True)
# reordena as colunas
colunas = ["accuracy", "precision", "recall", "f1-score", "AUC"]
resultado_CNN = resultado_CNN[colunas]
resultado_CNN_hinge = resultado_CNN_hinge[colunas]

In [21]:
display(resultado_CNN_hinge)
display(resultado_CNN)
display(resultado_linear)
display(resultado_poli)
display(resultado_rbf)

,accuracy,precision,recall,f1-score,AUC
0,1.000000,1.0,1.000000,1.000000,1.000000
1,1.000000,1.0,1.000000,1.000000,1.000000
2,1.000000,1.0,1.000000,1.000000,1.000000
3,1.000000,1.0,1.000000,1.000000,1.000000
4,1.000000,1.0,1.000000,1.000000,1.000000
5,1.000000,1.0,1.000000,1.000000,1.000000
6,1.000000,1.0,1.000000,1.000000,1.000000
7,1.000000,1.0,1.000000,1.000000,1.000000
8,1.000000,1.0,1.000000,1.000000,1.000000
9,1.000000,1.0,1.000000,1.000000,1.000000


,accuracy,precision,recall,f1-score,AUC
0,0.437500,0.437500,1.000000,0.608696,0.936508
1,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000
4,0.510417,0.000000,0.000000,0.000000,0.754668
5,1.000000,1.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,1.000000,1.000000,1.000000
7,1.000000,1.000000,1.000000,1.000000,1.000000
8,1.000000,1.000000,1.000000,1.000000,1.000000
9,1.000000,1.000000,1.000000,1.000000,1.000000


,accuracy,precision,recall,f1-score,AUC
0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0


,accuracy,precision,recall,f1-score,AUC
0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0


,accuracy,precision,recall,f1-score,AUC
0,1.00000,1.0,1.000000,1.000000,1.000000
1,0.84375,1.0,0.666667,0.800000,0.833333
2,1.00000,1.0,1.000000,1.000000,1.000000
3,1.00000,1.0,1.000000,1.000000,1.000000
4,1.00000,1.0,1.000000,1.000000,1.000000
5,1.00000,1.0,1.000000,1.000000,1.000000
6,1.00000,1.0,1.000000,1.000000,1.000000
7,1.00000,1.0,1.000000,1.000000,1.000000
8,1.00000,1.0,1.000000,1.000000,1.000000
9,1.00000,1.0,1.000000,1.000000,1.000000


In [22]:
resultados_finais = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1-score", "AUC"])
resultados_finais.loc["CNN"] = resultado_CNN.mean()
resultados_finais.loc["CNN hinge"] = resultado_CNN_hinge.mean()
resultados_finais.loc["CSVM linear"] = resultado_linear.mean()
resultados_finais.loc["CSVM poli"] = resultado_poli.mean()
resultados_finais.loc["CSVM rbf"] = resultado_rbf.mean()
resultados_finais

,accuracy,precision,recall,f1-score,AUC
CNN,0.968542,0.958542,0.979643,0.965141,0.979846
CNN hinge,0.961250,0.920000,0.917917,0.918901,0.964613
CSVM linear,1.000000,1.000000,1.000000,1.000000,1.000000
CSVM poli,1.000000,1.000000,1.000000,1.000000,1.000000
CSVM rbf,0.995625,1.000000,0.990476,0.994462,0.995238


In [23]:
resultados_finais.to_excel(f"Resultado teste sanidade {n_samples} samples {repeticoes} repeticoes.xlsx")